# demo_noaa_constituents -- Example Pytides Usage

Based loosely on the example by Sam Cox on the Pytides wiki, [How to use the NOAA's published Harmonic Constituents in Python with Pytides](https://github.com/sam-cox/pytides/wiki/How-to-use-the-NOAA's-published-Harmonic-Constituents-in-Python-with-Pytides).

This example uses the [NOAA constituents published for King's Point, NY](https://tidesandcurrents.noaa.gov/harcon.html?unit=0&timezone=0&id=8516945&name=Kings+Point&state=NY). These have been typed into this notebook below. 

Code at the bottom of this notebook illustrates how to download the harmonic consituents directly, a better approach in general since it is easier, less error-prone, and provides more digits of precision than the published tables.

To find the station number and the webpages for other stations, see [this NOAA page](https://tidesandcurrents.noaa.gov/stations.html?type=Water%20Levels).  After going to one of the station webpages, you will find links to the harmonic constituents and datums near the bottom of the page.  Make sure you select "meters" as the units and the desired time zone (local or GMT) and then refresh the page if necessary.

For more about tidal constituents, see for example:
 - [wikipedia page](https://en.wikipedia.org/wiki/Theory_of_tides#Tidal_constituents)
 - [NOAA page](https://tidesandcurrents.noaa.gov/about_harmonic_constituents.html)

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

### Import local submodule version of Pytides

The clawpack version includes some fixes to the original code needed to make it work in Python3.

In [ ]:
# Here's what we'd like to do:  (?)
#from clawpack.pytides.tide import Tide
#import clawpack.pytides.constituent as cons
#

# For now, hardwire in the path...
import sys, os
CLAW = os.environ['CLAW']  # path to Clawpack files
pathstr = os.path.join(CLAW, 'tidal-examples/pytides')
assert os.path.isdir(pathstr), '*** Need clawpack/tidal-examples/pytides ***'
print('Using Pytides from: %s' % pathstr)
if pathstr not in sys.path:
    sys.path.insert(0,pathstr)
    
from pytides.tide import Tide
import pytides.constituent as cons

Here are the NOAA constituents, in the order presented on their website for this particular station.

We omit the Z0 component, which is 0 relative to MSL and will be adjusted below to present results relative to a different datum, e.g. MLLW.

In [ ]:
constituents = [c for c in cons.noaa if c != cons._Z0]

#Phases and amplitudes (relative to GMT and in degrees and metres)
published_phases = [115.7,140.7,92.6,192,145.5,220.6,159.9,202.8,152.3,\
                    117.2,92,0,0,69.7,224.5,141.7,121.9,\
                    228.4,252.1,0,60.1,135.5,0,0,204.5,212.2,112.3,\
                    141.8,249.1,211.1,75.1,181.4,140.4,202.4,141.8,155,160.9]

published_amplitudes = [1.142,0.189,0.241,0.1,0.036,0.066,0.08,0.01,0.004,\
                        0.022,0.052,0,0,0.03,0.007,0.025,0.009,\
                        0.005,0.008,0,0.024,0.065,0,0,0.004,0.017,0.015,\
                        0.002,0.002,0.032,0.003,0.007,0.07,0.009,0.053,\
                        0.007,0.008]


Print out the constintuents for easy comparison with the [NOAA constituents page for station 8516945, Kings Point, NY](https://tidesandcurrents.noaa.gov/harcon.html?unit=0&timezone=0&id=8516945&name=Kings+Point&state=NY). Note that some of the names are slightly different from the NOAA names:

In [ ]:
print('#     Name      Amplitude   Phase')
for k,c in enumerate(constituents):
    print('%s   %s   %.3f     %7.3f' \
          % (str(k+1).ljust(4), c.name.ljust(7), published_amplitudes[k], published_phases[k]))

We can add a constant offset.  The published constituents are relative to MSL  (not MTL as stated in the Pytides wiki example).  Here we set the offset so that the plots will be relative to MLLW instead.  These values can be found on the [NOAA datums page for this station](https://tidesandcurrents.noaa.gov/datums.html?datum=STND&units=1&epoch=0&id=8516945&name=Kings+Point&state=NY).  Note that these values are relative to the station datum (STND) although the offset computed should be the same as long as the values of both `MSL` and `MLLW` used are relative to the same datum.  Also make sure `meters` is selected when looking at datums (and at constituents).

In [ ]:
MSL = 5.113
MLLW = 3.927
offset = MSL - MLLW
constituents.append(cons._Z0)
published_phases.append(0)
published_amplitudes.append(offset)

### Build the model, and a tide instance:

In [ ]:
assert(len(constituents)==len(published_phases)==len(published_amplitudes))
model = np.zeros(len(constituents), dtype = Tide.dtype)
model['constituent'] = constituents
model['amplitude'] = published_amplitudes
model['phase'] = published_phases

#Build a TIDE INSTANCE called tide from the MODEL called model
tide = Tide(model=model,radians=False)

In [ ]:
print('Predicted tide on January 1, 2013 relative to MLLW...')
print('    at 00:00 GMT: %.3fm\n    at 06:00 GMT: %.3fm' \
        % tuple(tide.at([datetime(2013,1,1,0,0,0), datetime(2013,1,1,6,0,0)])))


The [actual NOAA prediction](https://tidesandcurrents.noaa.gov/waterlevels.html?id=8516945&units=metric&bdate=20130101&edate=20130102&timezone=GMT&datum=MLLW&interval=6&action=) for 0000 and 0600 GMT on January 1 2013 are -0.079m and  2.206m relative to MLLW.

## Produce plots over a time range:

We can produce plots similar to the [actual NOAA prediction](https://tidesandcurrents.noaa.gov/waterlevels.html?id=8516945&units=metric&bdate=20130101&edate=20130102&timezone=GMT&datum=MLLW&interval=6&action=) over a couple of days.

In [ ]:
prediction_t0 = datetime(2013,1,1,0,0,0)
prediction_end= datetime(2013,1,3,0,0,0)
hrs=((prediction_end - prediction_t0).total_seconds())/3600.
print ('The data started at datetime: ',prediction_t0)
print ('The data ended at datetime: ',prediction_end)
print ('The data spanned %5i hours' %int(hrs))
print (' ')
hours = 0.1*np.arange(int(hrs)*10)
times = Tide._times(prediction_t0, hours)

### Evaluate the tide instance at the specified times

In [ ]:
my_prediction = tide.at(times)

### Find the high tides and low tides, and print out a tide table:

In [ ]:
# Find the highs and lows using tide.extrema generator function
# of the tidal instance called tide.  Save ext_hrs and ext_hts
# as numpy arrays for plotting later.

ext_vals=[t for t in tide.extrema(prediction_t0,prediction_end)]
print ('High and Low tides, relative to MLLW: ')

n_ext=len(ext_vals)
ext_hts=[]; ext_hilo=[]; ext_hrs=[]; ext_datetimes=[];
for i in range(n_ext):
    ext_tuple=ext_vals[i]
    ext_datetimes.append(ext_tuple[0])
    ext_hts.append(ext_tuple[1])
    ext_hilo.append(ext_tuple[2])
    ext_hrs.append( ((ext_tuple[0] - prediction_t0).total_seconds())/3600.)
ext_hrs=np.array(ext_hrs)
ext_hts=np.array(ext_hts)

#Print the extrema information
print (' ')
print (' Date        time        Hrs   Elevation      Hi-Low ') 
for i in range(n_ext):
    print ('%s %8.3f %8.3f m %8s ' %\
          (ext_datetimes[i].strftime('%Y-%m-%d  %H:%M:%S'), ext_hrs[i],ext_hts[i],ext_hilo[i]) )

## Plot the predicted tide

In [ ]:
titlestr = 'January 1-2, 2013 Example Tides \n' +\
            'Kings Point, NY (Station 8516945)'

plt.figure(figsize=(13,6))
plt.plot(hours, my_prediction, label="The data (38 NOAA)")
plt.plot(ext_hrs,ext_hts,'ro',label="Extrema")
plt.xticks(np.arange(0,49,12))
plt.xlabel('Hours since ' + str(prediction_t0) + '(GMT)')
plt.ylabel('Meters above MLLW')
plt.axis([-1,49,-1,4])
plt.legend(loc='upper left')
plt.grid(True)
plt.title(titlestr);

## Download harmonic consituents

Rather than typing in the harmonic constituents as done above, it is much easier and less prone to error to download them directly from the NOAA website.  The code below should produce the same constituents as used above.

We use a function in the GeoClaw `tidetools` module to do this.  Eventually this will be moved to geoclaw, but is local to this repository for development purposes...

In [ ]:
pathstr = os.path.abspath('..')
if pathstr not in sys.path:
    sys.path.insert(0,pathstr)
import tidetools

print('Using tidetools from: %s' % tidetools.__file__)

In [ ]:
station = 8516945 # Kings point, NY
print('Fetching harmonic constituents for station %s, standard 37 -- no Z0' % station)
harcon, harcon_info = tidetools.fetch_harcon(station, units='meters', verbose=False)

numbers = list(range(1,38))  
harcon_numbers = [h['number'] for h in harcon]
# make sure there are the expected number and in the right order:
assert harcon_numbers == numbers, \
      '*** unexpected harcon_numbers = %s' % numbers

Note that `harcon` is now a dictionary with keys such as `number`, `name`, `amplitude`, etc.
    
Print out these constituents, same as above but with names that agree with the [NOAA page](https://tidesandcurrents.noaa.gov/harcon.html?unit=0&timezone=0&id=8516945&name=Kings+Point&state=NY), and with more digits of precision in the amplitudes:

In [ ]:
print('#     Name      Amplitude     Phase')
for k,h in enumerate(harcon):
    print('%s   %s   %.5f     %9.4f' \
          % (str(h['number']).ljust(4), h['name'].ljust(7), h['amplitude'], h['phase_GMT']))

In [ ]:
print("Note that:  harcon_info['units'] = %s" % harcon_info['units'])

But our `tidetools` function converted the units to meters in computing `harcon`, since that's what we requested above.

## Translate harcon into a pytides model:

We can translate the `harcon` dictionary into the model needed by pytides as follows:

In [ ]:
NOAA_constituents = [c for c in cons.noaa if c != cons._Z0]

#Set the amplitudes and phases lists that will be needed for Pytides
NOAA_amplitudes   = [h['amplitude'] for h in harcon]   #in meters

#These are relative to GMT (0 deg West time meridan)
NOAA_phases_GMT   = [h['phase_GMT'] for h in harcon]  


In [ ]:
MSL = 5.113
MLLW = 3.928
offset = MSL - MLLW
NOAA_constituents.append(cons._Z0)
NOAA_phases_GMT.append(0)
NOAA_amplitudes.append(offset)

In [ ]:
assert(len(constituents) == len(NOAA_phases_GMT) \
        == len(NOAA_amplitudes))
NOAA_model = np.zeros(len(NOAA_constituents), dtype = Tide.dtype)
NOAA_model['constituent'] = NOAA_constituents
NOAA_model['amplitude'] = NOAA_amplitudes
NOAA_model['phase'] = NOAA_phases_GMT

#Build a TIDE INSTANCE called tide from the MODEL called model
NOAA_tide = Tide(model=NOAA_model,radians=False)

In [ ]:
print('Predicted tide on January 1, 2013 relative to MLLW...')
print('    at 00:00 GMT: %.3fm\n    at 06:00 GMT: %.3fm' \
        % tuple(NOAA_tide.at([datetime(2013,1,1,0,0,0), datetime(2013,1,1,6,0,0)])))

This is slightly different from the values obtained earlier because `harcon` has more digits in the constituents than what appears on the NOAA website and at the top of the notebook.